# Ensuring Consistency

**Activity Overview**: Ensure consistency by identifying and resolving conflicting values across datasets.

## Title: Customer Address Discrepancies

**Task**: Address customer address mismatches between CRM and marketing databases.

**Steps**:
1. Compare customer addresses in the CRM with those in the marketing database.
2. Identify records with conflicting address information.
3. Propose a method to consolidate records with verified addresses.

In [1]:
# Write your code from here

In [2]:
import pandas as pd

def find_address_mismatches(crm_file, marketing_file, customer_id_col='customer_id',
                            address_col_crm='address', address_col_marketing='address'):
    """
    Compares customer addresses between CRM and marketing databases to find mismatches.

    Args:
        crm_file (str): Path to the CRM database CSV file.
        marketing_file (str): Path to the marketing database CSV file.
        customer_id_col (str, optional): Common column for customer identification.
                                         Defaults to 'customer_id'.
        address_col_crm (str, optional): Address column name in the CRM file.
                                          Defaults to 'address'.
        address_col_marketing (str, optional): Address column name in the marketing file.
                                                Defaults to 'address'.

    Returns:
        pandas.DataFrame: A DataFrame containing records with conflicting addresses,
                          showing customer ID and addresses from both databases.
                          Returns None if files are not found.
    """
    try:
        crm_df = pd.read_csv(crm_file)
        marketing_df = pd.read_csv(marketing_file)
    except FileNotFoundError as e:
        print(f"Error: File not found: {e}")
        return None

    # Check if the required columns exist in both DataFrames
    if customer_id_col not in crm_df.columns or address_col_crm not in crm_df.columns:
        print(f"Error: '{customer_id_col}' or '{address_col_crm}' column missing in CRM data.")
        return None
    if customer_id_col not in marketing_df.columns or address_col_marketing not in marketing_df.columns:
        print(f"Error: '{customer_id_col}' or '{address_col_marketing}' column missing in marketing data.")
        return None

    # Merge the two DataFrames based on the customer ID
    merged_df = pd.merge(crm_df[[customer_id_col, address_col_crm]],
                         marketing_df[[customer_id_col, address_col_marketing]],
                         on=customer_id_col, suffixes=('_crm', '_marketing'), how='inner')

    if merged_df.empty:
        print("No matching customer IDs found between the CRM and marketing databases.")
        return pd.DataFrame()

    # Identify records with conflicting addresses
    mismatched_addresses_df = merged_df[merged_df[f'{address_col_crm}_crm'] != merged_df[f'{address_col_marketing}_marketing']]

    if not mismatched_addresses_df.empty:
        print("Customer address mismatches found:")
        return mismatched_addresses_df
    else:
        print("No customer address mismatches found between the CRM and marketing databases.")
        return pd.DataFrame()

# --- Proposing a Method to Consolidate Records with Verified Addresses ---

def consolidate_addresses(mismatched_df, crm_df, marketing_df, customer_id_col='customer_id',
                         address_col_crm='address', address_col_marketing='address',
                         source_preference='crm'):
    """
    Proposes a method to consolidate records with conflicting addresses,
    prioritizing addresses from a preferred source.
    Args:
        mismatched_df (pandas.DataFrame): DataFrame of records with conflicting addresses
                                           (output of find_address_mismatches).
        crm_df (pandas.DataFrame): The original CRM DataFrame.
        marketing_df (pandas.DataFrame): The original marketing DataFrame.
        customer_id_col (str, optional): Common column for customer identification.
                                         Defaults to 'customer_id'.
        address_col_crm (str, optional): Address column name in the CRM file.
                                          Defaults to 'address'.
        address_col_marketing (str, optional): Address column name in the marketing file.
                                                Defaults to 'address'.
        source_preference (str, optional): Preferred source for the verified address ('crm' or 'marketing').
                                           Defaults to 'crm'.

    Returns:
        pandas.DataFrame: A DataFrame showing the consolidated addresses based on the preference.
    """
    if mismatched_df.empty:
        print("No address mismatches to consolidate.")
        return pd.DataFrame()

    consolidated_addresses = []
    for index, row in mismatched_df.iterrows():
        customer_id = row[customer_id_col]
        address_crm = row[f'{address_col_crm}_crm']
        address_marketing = row[f'{address_col_marketing}_marketing']

        if source_preference.lower() == 'crm':
            verified_address = address_crm
            source = 'CRM'
        elif source_preference.lower() == 'marketing':
            verified_address = address_marketing
            source = 'Marketing'
        else:
            verified_address = None
            source = 'Manual Review Needed'
            print(f"Warning: Invalid source preference '{source_preference}'. Manual review suggested for customer ID: {customer_id}")

        consolidated_addresses.append({customer_id_col: customer_id,
                                       'address_crm': address_crm,
                                       'address_marketing': address_marketing,
                                       'verified_address': verified_address,
                                       'source_of_truth': source})

    return pd.DataFrame(consolidated_addresses)

# --- Example Usage ---
# Create dummy CSV files
crm_data = {'customer_id': [1, 2, 3, 4, 5],
            'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
            'address': ['123 Main St', '456 Oak Ave', '789 Pine Ln', '101 Elm Rd', '222 Maple Dr'],
            'email': ['alice@crm.com', 'bob@crm.com', 'charlie@crm.com', 'david@crm.com', 'eve@crm.com']}
marketing_data = {'customer_id': [1, 2, 3, 5, 6],
                  'segment': ['A', 'B', 'A', 'C', 'B'],
                  'address': ['123 Main St', '456 Oak Ave.', '789 Pine Lane', '222 Maple Dr.', '333 Willow Way'],
                  'phone': ['111-111-1111', '222-222-2222', '333-333-3333', '555-555-5555', '666-666-6666']}

crm_df = pd.DataFrame(crm_data)
marketing_df = pd.DataFrame(marketing_data)

crm_df.to_csv('crm_data.csv', index=False)
marketing_df.to_csv('marketing_data.csv', index=False)

# 1. Compare customer addresses
address_mismatches = find_address_mismatches('crm_data.csv', 'marketing_data.csv')

if address_mismatches is not None and not address_mismatches.empty:
    print(address_mismatches)

    # 3. Propose a method to consolidate records with verified addresses
    # Assuming CRM has more verified addresses
    consolidated_report = consolidate_addresses(address_mismatches, crm_df, marketing_df, source_preference='crm')
    print("\nProposed address consolidation (prioritizing CRM):")
    print(consolidated_report)

    # Assuming Marketing has more recently updated addresses
    consolidated_report_marketing_priority = consolidate_addresses(address_mismatches, crm_df, marketing_df, source_preference='marketing')
    print("\nProposed address consolidation (prioritizing Marketing):")
    print(consolidated_report_marketing_priority)

elif address_mismatches is not None:
    print("No address mismatches found, no consolidation needed.")

Customer address mismatches found:
   customer_id   address_crm address_marketing
1            2   456 Oak Ave      456 Oak Ave.
2            3   789 Pine Ln     789 Pine Lane
3            5  222 Maple Dr     222 Maple Dr.

Proposed address consolidation (prioritizing CRM):
   customer_id   address_crm address_marketing verified_address  \
0            2   456 Oak Ave      456 Oak Ave.      456 Oak Ave   
1            3   789 Pine Ln     789 Pine Lane      789 Pine Ln   
2            5  222 Maple Dr     222 Maple Dr.     222 Maple Dr   

  source_of_truth  
0             CRM  
1             CRM  
2             CRM  

Proposed address consolidation (prioritizing Marketing):
   customer_id   address_crm address_marketing verified_address  \
0            2   456 Oak Ave      456 Oak Ave.     456 Oak Ave.   
1            3   789 Pine Ln     789 Pine Lane    789 Pine Lane   
2            5  222 Maple Dr     222 Maple Dr.    222 Maple Dr.   

  source_of_truth  
0       Marketing  
1       M